# 사용시, train/demo test 는 제외하고 돌리면 됩니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 네이버 크롤링

In [ ]:
import requests 
from bs4 import BeautifulSoup 
from datetime import datetime 
import pandas as pd 

# 뉴스 언론사 선택
news_company = ['중앙일보', '연합뉴스', '조선일보', '국민일보']


# USER-AGENT : your user-agent
# https://www.whatismybrowser.com/detect/what-is-my-user-agent에서 user-agent의 값 가져오기
headers = {"User-Agent": "https://developers.whatismybrowser.com/useragents/parse/?analyse-my-user-agent=yes"}


#네이버 뉴스 랭킹 사이트
url = "https://news.naver.com/main/ranking/popularDay.naver"


# 서버 접속, html 해석
res = requests.get(url, headers = headers)
soup = BeautifulSoup(res.text, "lxml")


# 언론사별 1 ~ 5위 데이터 담긴 div 모두 가져오기
all_box = soup.find_all("div", attrs = {"class":"rankingnews_box"})


# 각 언론사별 1 ~ 5위 데이터 접근, lst_all_data에 저장
lst_all_data = []
# count = 1 
for box in all_box:
    company_name = box.strong.text
    lst_all_rank = box.find_all("li")
    num_rank = 1
    for rank in lst_all_rank:
        lst_data = []
        lst_data.append(datetime.now().strftime("%Y-%m-%d"))
        lst_data.append(company_name)
        if rank.a == None:
                continue
        lst_data.append(num_rank)
        lst_data.append(rank.a.text)
        lst_data.append(rank.a["href"])
        num_rank += 1
        lst_all_data.append(lst_data)
    # 몇회째 실시 되고 있는지 확인
    # print(f"전체 {str(len(all_box))} 중 {count}회 종료")
    # count += 1


df = pd.DataFrame(lst_all_data, columns = ["date", "company", "rank", "title", "link"])


# 선택된 언론사의 뉴스만 남기고, index 정렬
df = df[ df['company'].isin(news_company)].reset_index()

In [ ]:
import re
from tqdm import tqdm
import sys

final_urls = []
news_titles = []
news_contents = []
news_dates = []


for i in range(len(df)):
  final_urls.append(df['link'][i])

# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    content = " ".join(re.split(r"\s+", content))
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern = pattern1, repl = '', string = str(news_date))

    news_dates.append(news_date)

100%|██████████| 20/20 [00:08<00:00,  2.37it/s]


In [ ]:
news_df = pd.DataFrame({'date': news_dates,'title': news_titles,'link': final_urls,'content': news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
#now = datetime.now() 
#news_df.to_excel('{}.xlsx'.format(now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

중복 제거 후 행 개수:  20


In [ ]:
news_df.head()

,date,title,link,content
0,None,"""절인 배추서 역한 냄새, 다 버렸다""…김장 고무호스 주의보",https://n.news.naver.com/article/025/000324147...,[ 주부 구모(44)씨는 지난 26일 배추를 절일 때 마당에 설치된 수도꼭지에서 고...
1,None,"황의조에 ""한국말 해라"" 그는 통역사 아니었다…통역 논란 진실",https://n.news.naver.com/article/025/000324152...,[ 2022 카타르 월드컵 대표팀 공격수 황의조가 14일(현지시간) 카타르 도하 알...
2,None,"""압색 심정 느껴봐라""…한동훈 도곡 아파트 문 두드린 '더탐사'",https://n.news.naver.com/article/025/000324151...,[ 유튜브 '더탐사' 관계자 5명이 27일 한동훈 법무부 장관 아파트 현관 앞까지 ...
3,None,"""2명 탄다"" 신고했는데 사망자 5명…양양 추락 헬기 무슨 일",https://n.news.naver.com/article/025/000324152...,[ 27일 강원 양양군 현북면 명주사 인근에서 헬기 1대가 추락하는 사고가 발생해 ...
4,None,"이승기에게 무이자로 47억 빌린 대표 ""한남더힐 매입 무관""",https://n.news.naver.com/article/025/000324145...,[ 가수 이승기. 사진 후크엔터테인먼트 권진영 후크엔터테인먼트(이하 후크엔터) 대표...


# Train 준비

In [ ]:
%cd '/content/drive/MyDrive/nlp/summer/bart'

!git clone https://github.com/seujung/KoBART-summarization.git

!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

!pip install -r requirements.txt

%cd '/content/drive/MyDrive/nlp/summer/bart/data'

!tar -zxvf train.tar.gz

!tar -zxvf test.tar.gz

%cd ../

In [ ]:
!pip install torchtext==0.10.1

In [ ]:
!pip install pytorch-lightning==1.5.2

In [ ]:
!pip install torchvision==0.10.1+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

In [ ]:
!pip install torchmetrics==0.6.0

In [ ]:
# Train
# cpu
# !python train.py  --gradient_clip_val 1.0 --max_epochs 50 --default_root_dir logs  --batch_size 4 --num_workers 4

# gpu # batch_size 원래 10이었으나 많다고 해서 줄였음 # epoch 수 늘려서 강화 # epoch 당 1시간40분정도 잡고 하면 된다.
# !python train.py --gradient_clip_val 1.0 --max_epochs 3 --default_root_dir logs --gpus 1 --batch_size 5 --num_workers 4

In [ ]:
!python get_model_binary.py --hparams '/content/drive/MyDrive/nlp/summer/bart/logs/tb_logs/default/version_2/hparams.yaml' --model_binary '/content/drive/MyDrive/nlp/summer/bart/logs/model_chp/epoch=02-val_loss=1.363.ckpt'

In [ ]:
# demo test
# !streamlit run infer.py

# test - 직접 train

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained('./kobart_summary')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')
test1 = pd.read_excel('/content/drive/MyDrive/nlp/summer/sample/20221127_19시59분05초.xlsx') # 뉴스 excel
result_lst = []

for i in test1['content'].replace('@','').replace("'",""):
  if i:
      input_ids = tokenizer.encode(i[:2350])
      input_ids = torch.tensor(input_ids)
      input_ids = input_ids.unsqueeze(0)
      output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=3)
      output = tokenizer.decode(output[0], skip_special_tokens=True)
      result_lst.append(output)

len(result_lst)

In [ ]:
for i in result_lst:
  print(i)

# test2 - model import

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

In [ ]:
result_lst2 = []

for i in test1['content']:
  if i:
    text = i

    text = text.replace('\n', ' ').replace('@','').replace("'","")[:2300]
    print(text[:5])
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

    summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
    temp1 = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

    result_lst2.append(temp1)

In [ ]:
for i in result_lst2:
  print(i)
  print("==========================================================")